In [2]:
# Change to project's root directory
%cd "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard"

/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard


In [3]:
import os
import psycopg2
import pandas as pd
from google.colab import userdata

# Get credentials from Colab secrets.
db_user = userdata.get('RDS_USER')
db_pass = userdata.get('RDS_PASS')
db_host = userdata.get('RDS_HOST')
db_port = userdata.get('RDS_PORT')
db_name = userdata.get('RDS_NAME')

In [4]:
!pip install -q psycopg2-binary
!pip install -q dbt-postgres --use-deprecated=legacy-resolver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.5/86.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.9/985.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.7/442.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
# Ingest raw data and create the raw_saas_metrics table
!DB_HOST={db_host} DB_PORT={db_port} DB_NAME={db_name} DB_USER={db_user} DB_PASS={db_pass} python "scripts/database_setup.py"

Successfully connected to the database.
Table 'raw_saas_metrics' is ready.
Successfully ingested 5000 records into raw_saas_metrics.
Database connection closed.


In [7]:
# Run the dbt pipeline.
!DB_HOST={db_host} DB_PORT={db_port} DB_NAME={db_name} DB_USER={db_user} DB_PASS={db_pass} python "scripts/pipeline.py"

profiles.yml created successfully.
Creating core dbt project files...
Running the dbt pipeline...
08:30:37  Running with dbt=1.10.13
08:30:38  Registered adapter: postgres=1.9.1
08:30:40  Unable to do partial parsing because profile has changed
08:30:45  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.financial_forecasting.example
08:30:48  Found 2 models, 3 data tests, 1 source, 447 macros
08:30:48  
08:30:48  Concurrency: 1 threads (target='dev')
08:30:48  
08:30:54  1 of 2 START sql view model public.stg_raw_saas_metrics_data ................... [RUN]
08:31:01  1 of 2 OK created sql view model public.stg_raw_saas_metrics_data .............. [CREATE VIEW in 7.45s]
08:31:01  2 of 2 START sql view model public.fact_monthly_revenue ........................ [RUN]
08:31:06  2 of 2 OK created sql view model public.fact_monthly_revenue ................... [CREATE VIEW in 4.72s]
08:31:08  

--- Ran the pipeline and set up df. Now to create and train the forecasting model ---

In [8]:
# Making sure prophet is installed
!pip install -q prophet
!pip install -q dbt-postgres prophet pandas sqlalchemy psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [9]:
# Execute the forecasting pipeline.
!DB_HOST={db_host} DB_PORT={db_port} DB_NAME={db_name} DB_USER={db_user} DB_PASS={db_pass} python "scripts/run_forecasting.py"

Fetching clean data from fact_monthly_revenue...
/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard/scripts/run_forecasting.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  revenue_df = pd.read_sql(query, conn)
Training Prophet model and generating 24-month forecast...
08:35:38 - cmdstanpy - INFO - Chain [1] start processing
08:35:38 - cmdstanpy - INFO - Chain [1] done processing
Ingesting forecast into fact_monthly_revenue_forecast...
Forecast data successfully loaded.


--- Generate time-series forecast for 24 months ---

--- Clean the forecasted data and push to database ---

In [ ]:
from sqlalchemy import create_engine

# Clean the forecast dataframe.

# Keep only the relevant columns in a new df.
final_forecast_df = forecast[[
    'ds',            # The date
    'yhat',          # The point prediction (forecasted MRR)
    'yhat_lower',    # The lower bound of the prediction interval
    'yhat_upper'     # The upper bound of the prediction interval
]].copy()

# Rename the month and prediction columns for clarity in the database
final_forecast_df = final_forecast_df.rename(columns={'yhat': 'forecasted_mrr', 'ds': 'subscription_month'})

print("Forecast DataFrame cleaned and ready for ingestion.")
print(final_forecast_df.tail())


Forecast DataFrame cleaned and ready for ingestion.
   subscription_month  forecasted_mrr     yhat_lower     yhat_upper
55         2026-08-01   336810.642202  305517.824208  365591.841488
56         2026-09-01   336666.968282  307301.709088  362292.871389
57         2026-10-01   379952.174536  349614.756758  410489.490198
58         2026-11-01   370279.906376  343384.499341  399985.697669
59         2026-12-01   329600.829504  299612.139123  357613.444465


In [ ]:
# Ingest into PostgreSQL

# Create a SQLAlchemy engine for bulk insertion
# We set the connection variables at start of notebook
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

# Write the DataFrame to a new table.
# if_exists='replace': Recreates the table every time.
# index=False: Don't write the pandas index as a column.
final_forecast_df.to_sql(
    'fact_monthly_revenue_forecast',  # new table name
    engine,
    if_exists='replace',
    index=False
)

print("\nForecast data successfully loaded into 'fact_monthly_revenue_forecast' table.")


Forecast data successfully loaded into 'fact_monthly_revenue_forecast' table.
